<a href="https://colab.research.google.com/github/lupis30puc/yelp_bert_random_forest/blob/lupis30puc-update-1/Yelp_RF_mimic_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Yelp Polarity on kaggle](https://www.kaggle.com/yelp-dataset/yelp-dataset)

12,993 samples from the Yelp Dataset Challenge 2020. 
Divided on train, validation and test subsets. 
Their corresponding sizes are: 10,394 train samples, 1,949 validation samples and 650 test samples.


Tutorial on which I support: 
[Sentiment Analysis Yelp with Random Forest](https://www.kaggle.com/omkarsabnis/sentiment-analysis-on-the-yelp-reviews-dataset)

In [22]:
!pip install transformers

     |████████████████████████████████| 1.5MB 8.6MB/s 
     |████████████████████████████████| 2.9MB 37.7MB/s 
     |████████████████████████████████| 890kB 52.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=b251a508b4413014ba5fb24ff2a05d9eb64fe192c91a2080018387899458ec9b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
%matplotlib inline
import time

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


I want to create a boolean matrix with the input ids of the datasets...

In [49]:
import torch
test_dataset = torch.load('/content/drive/MyDrive/Yelp/sample_yelp_tensors/_test_150')
val_dataset = torch.load('/content/drive/MyDrive/Yelp/sample_yelp_tensors/_validation_150')
train_dataset = torch.load('/content/drive/MyDrive/Yelp/sample_yelp_tensors/_train_150')

In [50]:
len(train_dataset)

10394

In [11]:
len(test_dataset)

650

In [10]:
test_dataset[0][0].numpy()

array([  101,  2044,  3752,  2035,  1996,  2307,  4391,  2057,  2245,
        2057,  1040,  2507,  4012, 15455,  6072,  1037,  3046,  2005,
        6265,  2057,  4694,  1056,  9364,  2057,  2179,  1996, 25545,
        2465,  2100,  2007,  3737, 23127,  4406,  2070,  2060,  5171,
        4916,  7884,  2057,  2020, 11188, 22775,  1998,  3491,  2000,
        1037,  3332,  9065, 11772,  2048,  4127,  1997, 26509,  2020,
        5359, 13364,  1996, 12183,  2038,  2172,  2000,  5454,  2013,
        1045,  4900,  1996,  6265,  2569,  2007,  1037,  3601,  1997,
        2048,  3760,  4372, 13334,  2015,  1996, 26192,  2094,  7975,
        4372,  5428, 27266,  2050,  2007, 16709, 12901,  2001,  6581,
        2021,  2130,  2488,  2001,  1996,  7975,  9610,  7712, 18003,
        2050,  2009,  2001,  5642,  2011,  3903,  1997,  5785,  1998,
        2128, 13017, 13435,  2673,  2790,  4840,  1998, 25628,  1996,
        4825,  2003, 21121,  2011,  1045,  2903,  2155,  2372,  2013,
        3290,  2877,

In [54]:
test_index = []
test_ids = []
test_labels = []
for i in range(len(test_dataset)): 
    test_index.append(i)
    test_ids.append(test_dataset[i][0].numpy())
    test_labels.append(int(test_dataset[i][2]))
print(test_ids[2])
print(test_labels[2])

[  101  2200  4550  3309 14855 10841 13793  2003  2200  4121  6457  1998
  5723  2024  4121  1998  4550  3193  2001  3835  2348  2009  2001  1996
  2067  1997  1996  3309  9568  2001  3835  1998 27238  2001  2307  2085
  1996  2919  2694  2921 12809  2296  2261  2847  2005  2195  2781  2012
  1037  2051  2057  2020  2006  1996 16215  2723  1998  2657  1996  3345
  2296  2305  2035 27046  5833  1996  2305  5189  1998  4415 10047  1037
  8441  3345  3992  2005  1057  1052  1998  2008  2015  1996  2197  2518
  1045  2215 12447  2033  2039  2185  2013  2147  2061  2008  3084  6385
  1997  3714  3637  2035  2305  2196  2204  1045  2052  2025  2994  2182
  2153  2004  2057  2035  2113  1999  7136  2087  2111  2024  1999  2037
  3309  2069  2000  3637   102     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0]
0


In [55]:
print(len(test_index))

650


In [21]:
a = input_ids[0]
unique_elements, counts_elements = np.unique(a, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[  101   102  1037  1040  1045  1056  1996  1997  1998  1999  2000  2001
   2003  2005  2007  2009  2011  2013  2015  2020  2021  2026  2035  2038
   2044  2045  2048  2050  2057  2060  2069  2070  2094  2100  2128  2130
   2155  2172  2179  2245  2256  2307  2344  2372  2465  2488  2507  2569
   2653  2673  2790  2877  2903  3046  3290  3325  3332  3491  3601  3737
   3752  3760  3903  4012  4127  4372  4391  4406  4694  4825  4840  4900
   4916  4997  5171  5359  5428  5454  5642  5785  6072  6265  6581  7263
   7615  7712  7884  7975  8803  9065  9364  9610 11188 11772 12183 12901
  13017 13334 13364 13435 15455 16709 18003 20888 21121 22775 23127 25545
  25628 26192 26509 27266]
 [    1     1     5     1     2     1     7     3     3     1     3     4
      1     1     3     1     2     2     1     2     1     1     3     1
      1     1     2     2     5     1     1     1     1     1     1     1
      1     1     1     1     1     1  

In [57]:
type(unique_elements)

numpy.ndarray

## Otros:

In [ ]:
# NO NEED FOR THE DATASET OR YES?
#train_df = pd.read_pickle('/content/drive/MyDrive/Yelp/sample_train_10394.pkl')
#validation_df = pd.read_pickle('/content/drive/MyDrive/Yelp/sample_validation_1949.pkl')
#test_df = pd.read_pickle('/content/drive/MyDrive/Yelp/sample_test_650.pkl')

#sample_df = pd.read_pickle('/content/drive/MyDrive/Yelp/sample_yelp_reviews_12993.pkl')

#train_df.reset_index(drop=True, inplace=True)
#validation_df.reset_index(drop=True, inplace=True)
#test_df.reset_index(drop=True, inplace=True)
#train_df.head()

In [23]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/Yelp/model_save3/')
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [36]:
tokenizer.vocab_files_names

{'vocab_file': 'vocab.txt'}

In [34]:
tokenizer.vocab_size

30522

In [46]:
list(tokenizer.vocab.keys())[1996] #first actual word in vocab

'the'

In [48]:
list(tokenizer.vocab.keys())[29611] #last actual word in vocab

'necessitated'

In [47]:
for i in range(len(list(tokenizer.vocab.keys()))):
  if list(tokenizer.vocab.keys())[i] == 'necessitated':
    print(i)
    break

29611


## Continuación

In [56]:
# 27615 words in total.....
vocab = list(tokenizer.vocab.keys())[1996:29612]

In [67]:
# get a list of all unique ids
unique_ids = []
for i in test_index:
  uni_elements = np.unique(test_ids[i])
  for e in uni_elements: 
    if e not in unique_ids:
      unique_ids.append(e)


In [68]:
len(unique_ids)

5580

In [72]:
df1 = pd.DataFrame(index=test_index, columns=unique_ids)
df1.fillna(0, inplace=True)
df1

,101,102,1037,1040,1045,1056,1996,1997,1998,1999,2000,2001,2003,2005,2007,2009,2011,2013,2015,2020,2021,2026,2035,2038,2044,2045,2048,2050,2057,2060,2069,2070,2094,2100,2128,2130,2155,2172,2179,2245,...,10169,17395,18971,19717,19905,5249,17331,4063,5978,6184,6313,6862,9313,10415,10672,16663,18906,5150,6498,19873,3888,14890,17460,25006,25249,17093,9963,24494,2400,6077,6380,12394,15047,18846,7946,8304,20014,24436,26986,18152
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
646,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
648,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [81]:
%%time
for i in test_index:
  for id in unique_ids:
    if id in test_ids[i]:
      df1[id][i] = 1

CPU times: user 18.6 s, sys: 8.86 ms, total: 18.6 s
Wall time: 18.7 s


In [87]:
if 10169 in test_ids[0]:
  print(1)

In [88]:
df1.tail()

,101,102,1037,1040,1045,1056,1996,1997,1998,1999,2000,2001,2003,2005,2007,2009,2011,2013,2015,2020,2021,2026,2035,2038,2044,2045,2048,2050,2057,2060,2069,2070,2094,2100,2128,2130,2155,2172,2179,2245,...,10169,17395,18971,19717,19905,5249,17331,4063,5978,6184,6313,6862,9313,10415,10672,16663,18906,5150,6498,19873,3888,14890,17460,25006,25249,17093,9963,24494,2400,6077,6380,12394,15047,18846,7946,8304,20014,24436,26986,18152
645,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
646,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0
647,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0
648,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
649,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [89]:
df1.describe()

,101,102,1037,1040,1045,1056,1996,1997,1998,1999,2000,2001,2003,2005,2007,2009,2011,2013,2015,2020,2021,2026,2035,2038,2044,2045,2048,2050,2057,2060,2069,2070,2094,2100,2128,2130,2155,2172,2179,2245,...,10169,17395,18971,19717,19905,5249,17331,4063,5978,6184,6313,6862,9313,10415,10672,16663,18906,5150,6498,19873,3888,14890,17460,25006,25249,17093,9963,24494,2400,6077,6380,12394,15047,18846,7946,8304,20014,24436,26986,18152
count,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,650.0,...,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000
mean,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.003077,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.055427,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223
min,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

### Tr

In [ ]:
x = vocab.transform(train_x)
#Shape of the matrix:
print("Shape of the sparse matrix: ", x.shape)
#Non-zero occurences:
print("Non-Zero occurences: ",x.nnz)

# DENSITY OF THE MATRIX
density = (x.nnz/(x.shape[0]*x.shape[1]))*100
print("Density of the matrix = ",density)

Shape of the sparse matrix:  (10394, 39269)
Non-Zero occurences:  520479
Density of the matrix =  0.1275177546788985


In [ ]:
%%time
# CONVERTING THE WORDS INTO A VECTOR
vocab2 = CountVectorizer(analyzer=text_process).fit(test_x)
x2 = vocab.transform(test_x)
#Shape of the matrix:
print("Shape of the sparse matrix: ", x.shape)
#Non-zero occurences:
print("Non-Zero occurences: ",x.nnz)

# DENSITY OF THE MATRIX
density = (x.nnz/(x.shape[0]*x.shape[1]))*100
print("Density of the matrix = ",density)

Shape of the sparse matrix:  (10394, 39269)
Non-Zero occurences:  520479
Density of the matrix =  0.1275177546788985


In [ ]:
%%time
# CONVERTING THE WORDS INTO A VECTOR
vocab = CountVectorizer(analyzer=text_process).fit(sample_x)
x = vocab.transform(sample_x)
#Shape of the matrix:
print("Shape of the sparse matrix: ", x.shape)
#Non-zero occurences:
print("Non-Zero occurences: ",x.nnz)

# DENSITY OF THE MATRIX
density = (x.nnz/(x.shape[0]*x.shape[1]))*100
print("Density of the matrix = ",density)

Shape of the sparse matrix:  (12993, 44314)
Non-Zero occurences:  650798
Density of the matrix =  0.11303054400013844
CPU times: user 5min 27s, sys: 40.9 s, total: 6min 8s
Wall time: 6min 8s


In [ ]:
len(y_train)

10394

## Training

In [92]:
# SPLITTING THE DATASET INTO TRAINING SET AND TESTING SET
# SAME RANDOM STATE AS IN BERT....
x_train,x_test,y_train,y_test = train_test_split(df1,test_labels,test_size=0.2,random_state=42)

In [93]:
%%time
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rmfr = RandomForestClassifier()

CPU times: user 77 µs, sys: 1e+03 ns, total: 78 µs
Wall time: 82.7 µs


In [94]:
%%time
rmfr.fit(x_train,y_train)

CPU times: user 635 ms, sys: 7.98 ms, total: 643 ms
Wall time: 648 ms


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [95]:
%%time
predrmfr = rmfr.predict(x_test)
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test,predrmfr))
print("Score:",round(accuracy_score(y_test,predrmfr)*100,2))
print("Classification Report:")
print(classification_report(y_test,predrmfr))

Confusion Matrix for Random Forest Classifier:
[[47 16]
 [22 45]]
Score: 70.77
Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.75      0.71        63
           1       0.74      0.67      0.70        67

    accuracy                           0.71       130
   macro avg       0.71      0.71      0.71       130
weighted avg       0.71      0.71      0.71       130

CPU times: user 32.3 ms, sys: 2.01 ms, total: 34.3 ms
Wall time: 41.9 ms


In [96]:
importance = rmfr.feature_importances_

In [97]:
len(importance)

5580

In [98]:

feature_importance = pd.DataFrame({'keys': unique_ids, 'imp': importance})

In [99]:
feature_importance.head()

,keys,imp
0,101,0.0
1,102,0.0
2,1037,0.0
3,1040,0.0
4,1045,0.0


In [109]:
top10 = list(feature_importance.sort_values(by=['imp'], ascending=False)['keys'][:10])

In [110]:
for word in top10:
  print(list(tokenizer.vocab.keys())[word])

not
horrible
worst
delicious
don
good
asked
no
poor
definitely
